In [1]:
# loading common tools, plotting, bson, progress bar, etc
import os, sys, math, io
import numpy as np
import pandas as pd
import multiprocessing as mp
import bson
import struct
from PIL import Image
import time
import shutil

%matplotlib inline
import matplotlib.pyplot as plt

from collections import defaultdict
from tqdm import *

In [2]:
# loading PyTorch
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.utils.data import Dataset

In [3]:
###  Check the content of `sample_submission.csv` file
data_dir = '/media/hua/HuaSSD/KaggleData/CdiscountImageClassificationChallenge/' #data saved in SSD
check_sub_df = pd.read_csv(data_dir + "sample_submission.csv")
check_sub_df.head()

,_id,category_id
0,10,1000010653
1,14,1000010653
2,21,1000010653
3,24,1000010653
4,27,1000010653


In [4]:
###  create submission dataframe
sub_df = pd.DataFrame(columns=['_id','category_id'])
sub_df.head()

,_id,category_id


In [5]:
# load categories id from category_names.csv
cat_df = pd.read_csv(data_dir + 'category_names.csv', index_col="category_id")
# Maps the category_id to an integer index.
cat_df["category_idx"] = pd.Series(range(len(cat_df)), index=cat_df.index)
cat_df.tail()

,category_level1,category_level2,category_level3,category_idx
category_id,,,,
1000003412,VIN - ALCOOL - LIQUIDES,JUS - SODA -SIROP-BOISSON LACTEE,SODA-THE GLACE,5265
1000003530,VIN - ALCOOL - LIQUIDES,VIN,ASSORTIMENT VIN,5266
1000003402,VIN - ALCOOL - LIQUIDES,VIN,VIN BLANC,5267
1000003404,VIN - ALCOOL - LIQUIDES,VIN,VIN ROSE,5268
1000003400,VIN - ALCOOL - LIQUIDES,VIN,VIN ROUGE,5269


In [6]:
# ok, hash map for id(10000, and random...) and idx(0...5269)
id_idx = zip(cat_df.index, cat_df['category_idx'])
idx_id = zip(cat_df['category_idx'], cat_df.index)
cat2idx = dict(id_idx)
idx2cat = dict(idx_id)

In [7]:
# here we start play with model, resnet 101, 49 big cat, 483 middle cat, and 5270 small cat
model = torchvision.models.resnet101(pretrained=False)
model.avgpool = nn.AvgPool2d(kernel_size = 6)
model.fc = nn.Linear(in_features=2048, out_features=49 + 483 + 5270)

In [10]:
# loading models ...
model_dir = '/media/hua/HuaSSD/KaggleData/CdiscountImageClassificationChallenge/ModelResNet101/' #model saved in SSD
# em... we need to change this from time to time
trained_model = model_dir + 'ResNet101_L4L5_4Epoch_DO05_W111_20171204.pth.tar'

def load_model(model, trained_model):
    if os.path.isfile(trained_model):
        print("=> loading checkpoint '{}'".format(trained_model))
        checkpoint = torch.load(trained_model)
        model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}'".format(trained_model))
        return model
    else:
        print("=> no checkpoint found at '{}'".format(best_model))

In [11]:
model = load_model(model, trained_model)

=> loading checkpoint '/media/hua/HuaSSD/KaggleData/CdiscountImageClassificationChallenge/ModelResNet101/ResNet101_L4L5_4Epoch_DO05_W111_20171204.pth.tar'
=> loaded checkpoint '/media/hua/HuaSSD/KaggleData/CdiscountImageClassificationChallenge/ModelResNet101/ResNet101_L4L5_4Epoch_DO05_W111_20171204.pth.tar'


In [12]:
### change the fc layer back to the one without levelID to save time in evaluation.
new_fc = nn.Linear(in_features=2048, out_features=5270)
new_fc.weight.data = model.fc.weight.data[532:]
new_fc.bias.data = model.fc.bias.data[532:]
model.fc = new_fc

In [13]:
model.cuda() #load into gpu

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): Bottleneck (
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (downsample): Sequential (
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1): Bott

In [14]:
test_bson_path = os.path.join(data_dir, "test.bson")
test_bson_file = open(test_bson_path, "rb")
test_data = bson.decode_file_iter(open(test_bson_path, "rb"))
num_test_products = 1768182

In [15]:
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
#transform_train = T.Compose([T.RandomHorizontalFlip(), 
#                             T.ToTensor(),T.Normalize(mean=mean, std=std)])
transform_val = T.Compose([T.ToTensor(),T.Normalize(mean=mean, std=std)])
#transform_0 = T.Compose([T.ToTensor(),T.Normalize(mean=mean, std=std)])
#transform_1 = T.Compose([hflip, transform_0])
#transform_2 = T.Compose([T.Scale(size=240), T.RandomCrop(size=180), transform_0])
#transform_3 = T.Compose([T.Scale(size=240), T.RandomCrop(size=180), transform_1])

In [13]:
### Evaluate method without any average
model.eval()
with tqdm(total=num_test_products) as pbar:
    for c, d in enumerate(test_data):
        product_id = d["_id"]
        num_imgs = len(d["imgs"])

        batch_x = []
        for i in range(num_imgs):
            bson_img = d["imgs"][i]["picture"]

            # Load and preprocess the image.
            image = io.BytesIO(bson_img)
            img = Image.open(image)
            x = transform_val(img)

            # Add the image to the batch.
            batch_x.append(x)

        batch_x = torch.stack(dim=0, sequence=batch_x).cuda()
        img_var = Variable(batch_x, volatile=True)
        prediction = model(img_var).data
        avg_pred = prediction.mean(dim=0)
        _, cat_idx = avg_pred.max(dim=0)
        submission_df.iloc[c]["category_id"] = idx2cat[cat_idx[0]]   
        pbar.update()

submission_df.to_csv("my_submission.csv.gz", compression="gzip", index=False)

100%|██████████| 1768182/1768182 [8:56:25<00:00, 54.94it/s]  


In [14]:
submission_df.head()

,_id,category_id
0,10,1000005605
1,14,1000010653
2,21,1000010653
3,24,1000002334
4,27,1000022508


In [16]:
submission_df.head()

,_id,category_id
0,10,1000015802
1,14,1000010653
2,21,1000010653
3,24,1000002373
4,27,1000022508


### it seems that using average is not as good as without any augmentation.

In [14]:
submission_df.head()

,_id,category_id
0,10,1000005605
1,14,1000010653
2,21,1000010653
3,24,1000002334
4,27,1000003813
